# CSE 572: Lab 13

In this lab, you will practice implementing techniques for dimensionality reduction using Kernel PCA and autoencoder neural networks.

To execute and make changes to this notebook, click File > Save a copy to save your own version in your Google Drive or Github. Read the step-by-step instructions below carefully. To execute the code, click on each cell below and press the SHIFT-ENTER keys simultaneously or by clicking the Play button. 

When you finish executing all code/exercises, save your notebook then download a copy (.ipynb file). Submit the following **three** things:
1. a link to your Colab notebook,
2. the .ipynb file, and
3. a pdf of the executed notebook on Canvas.

To generate a pdf of the notebook, click File > Print > Save as PDF.

## Kernel PCA

Principal component analysis (PCA) is a classical method for reducing the dimensionality (number of attributes) in the data by projecting the data from its original high-dimensional space into a lower-dimensional space. The conventional formulation of PCA is limited to finding a linear projection of the data, so if the original data are not linearly separable then the PCA-projected data also will not be. Kernel PCA is a simple modification of PCA that uses kernel functions to project data into a higher-dimensional space where it may become linearly separable and compute PCA in the higher-dimensional space, using special kernel functions to make this computation feasible.

In this section, we will apply Kernel PCA to the same image dataset from Lab 11 and compare the results to linear PCA.

### Load the dataset

We will use a preprocessed subset of the “Labeled Faces in the Wild” (LFW) dataset, which you can read more about [here](http://vis-www.cs.umass.edu/lfw/).

In [ ]:
from sklearn.datasets import fetch_lfw_people

lfw_people = fetch_lfw_people(min_faces_per_person=70)

# data attribute gives the data matrix with the image dimension flattened
X = lfw_people.data
print('Num samples: {}'.format(X.shape[0]))
print('Num features: {}'.format(X.shape[1]))

# images attribute gives the unflattened image dimension
print('Image dimensions: {} x {}'.format(lfw_people.images.shape[1], lfw_people.images.shape[2]))

# the label to predict is the id of the person
y = lfw_people.target
# target_names attribute tells us the name of the person associated with each id
target_names = lfw_people.target_names
n_classes = target_names.shape[0]
print('Num classes: {}'.format(n_classes))
print('Class names:', target_names)

The dataset has 1,288 images of the faces of 7 different people. The code below prints the number of samples from each class.

In [ ]:
for i in range(n_classes):
    print('Number of instances in class {} ({}): {}'.format(i,
                                                            target_names[i],
                                                            y[y==i].shape[0]
                                                           ))

To get an idea of what is in our dataset, we visualize a random face from each class below. (Note: You can run this cell many times to see different random examples.)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, axes = plt.subplots(ncols=n_classes, figsize=(13,6))

for i in range(n_classes):
    rand_ind = np.random.randint(0, y[y==i].shape[0])
    axes[i].imshow(lfw_people.images[y==i][rand_ind], cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(target_names[i])

Next, we will split the data into training and test subsets, using 30\% of the data for testing.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

Next we standardize the data so that the mean of all attributes is 0 using the StandardScaler() object in scikit-learn to standardize the data. We fit the scaler to the training data and apply it to both the training and test data.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Kernel PCA

Kernel PCA uses one of several choices of kernel functions to transform the data into a higher-dimensinoal space and then compute the principal components (eigenvectors) of the data in the high-D space. We then use these principal components to transform the data into a new set of features that maximizes variance in the dataset. 

The KernelPCA class in scikit-learn gives four options for kernel functions (the same functions available for SVM): 'poly', 'rbf', 'sigmoid', and 'cosine'. In the code below, we implement KernelPCA using 10 principal components and the sigmoid kernel.

In [ ]:
from sklearn.decomposition import KernelPCA

n_components = 10

kpca = KernelPCA(n_components=n_components, kernel='sigmoid', fit_inverse_transform=True).fit(X_train)

We can use our PCA model (i.e., the principal component vectors found in our data) to transform our dataset into the new reduced-dimensional space. This is done in the below code using the `transform()` function.

In [ ]:
X_train_kpca = kpca.transform(X_train)
X_test_kpca = kpca.transform(X_test)

### SVM classifier with Kernel PCA

Now that we've reduced the dimensionality of our dataset from 2914 to 10, we can use these new features as our inputs for classification. In this example, we'll use a Support Vector Machine (SVM) classifier. In the cell below, train an SVM with `C=10000` and `kernel='rbf'`.

In [ ]:
from sklearn.svm import SVC

clf = SVC(C=10, kernel='rbf')
clf = clf.fit(X_train_kpca, y_train)

In the cell below, use the trained model to make predictions for the test set.

In [ ]:
y_pred = clf.predict(X_test_kpca)

The next cell shows how we can use the `classification_report()` function in sklearn to print several metrics computed for the test set. 

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=target_names))

We can also use the `ConfusionMatrixDisplay` object to visualize the confusion matrix for the test set predictions.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(
    clf, X_test_kpca, y_test, display_labels=target_names, xticks_rotation="vertical"
)

plt.tight_layout()
plt.show()

**Question 1: What is the difference between the covariance matrix and the confusion matrix? What are the dimensions of each in terms of the number of attributes (m) and the number of classes (c)? How are each used here?**

**Answer:**

YOUR ANSWER HERE

### Data reconstruction

We can reconstruct the original input images by multiplying the encoded data in the principal subspace by the principal component vectors. This reverse transformation is called the *inverse transform*. The code below reconstructs the original data using the KernelPCA `inverse_transform()` function.

In [ ]:
train_recon_kpca = kpca.inverse_transform(X_train_kpca)
test_recon_kpca = kpca.inverse_transform(X_test_kpca)

Create a plot to visualize the original image and reconstructed image for the first 5 images in the test set. Plot the original images on the first row and the reconstructions on the second row. Print the class name as the title for each original image and turn off the axis labels/tickmarks (as in the earlier face visualization from this lab).

In [ ]:
fig, axes = plt.subplots(ncols=5, nrows=2, figsize=(10,5))
n_images = 5

# YOUR CODE HERE

In Lab 11, we used linear PCA to reduce the data dimensionality and use the transformed data as the input to SVM as we did in Lab 12 with KernelPCA. However, in Lab 11 we did not reconstruct the original data from the principal subspace as we did above with KernelPCA. 

In the cell(s) below, fit a new linear PCA model with 10 principal components (same as was done in Lab 11). Then, plot the above visualization again, but this time add a third row for the same images reconstructed using linear PCA.

In [ ]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

## Autoencoder neural network

An autoencoder is a type of neural network in which the outputs are the same as the inputs. In other words, the goal of the network is to reconstruct the original input image by minimizing the mean squared error loss function. The autoencoder consists of two main components: an encoder network that maps the input data to a low dimensional representation $z$ and a decoder network that maps $z$ back to the original dimension. 

In the code below, we implement a simple autoencoder network using dense (feed-forward) layers and train it using our LFW faces dataset.

In [ ]:
import tensorflow as tf
seed = 256
tf.keras.utils.set_random_seed(seed)

import keras
from keras import layers

# This is the size of our encoded representations
encoding_dim = 32  

# This is our input image
input_img = keras.Input(shape=(X_train.shape[1],))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(128, activation='relu')(input_img)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(X_train.shape[1], activation='sigmoid')(decoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In addition to the full autoencoder model, we can construct the encoder and decoder models separately. Here we will construct the encoder model so that we can use it to reduce the data dimensionality without having to also reconstruct the data (which is done by `autoencoder`).

In [ ]:
# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

Next we compile the model. We'll use the Adam optimizer and mean squared error loss.

In [ ]:
autoencoder.compile(optimizer='adam', loss='mse')

Now we train our model. Below we train for 100 epochs using a batch size of 32. 

In [ ]:
autoencoder.fit(x=X_train, y=X_train,
                epochs=100,
                batch_size=32,
                shuffle=True)

**Question 2: In the code above, why did we set x and y both equal to `X_train`?**

**Answer:**

YOUR ANSWER HERE

Now we have our trained autoencoder for which the input is an image and the output is the reconstruction. Next we use our trained autoencoder to reconstruct the test images.

In [ ]:
test_recon_ae = autoencoder.predict(X_test)

In the cell below, visualize the same 5 as we did for KernelPCA and linear PCA. You can copy the same code as the previous code block used for visualization and add a fourth row for the autoencoder reconstructions.

In [ ]:
# YOUR CODE HERE

**Question 3: The reconstructions by the autoencoder do not look as accurate as the reconstructions by the PCA and KernelPCA models. This does not mean that linear or kernel PCA is inherently better than an autoencoder for reducing the dimensionality of this dataset---it is probable that if we spent time tuning all the hyperparameters of the autoencoder, we would find a model that has a better solution. However, there are many more hyperparameters to tune in the autoencoder compared to the PCA models. List at least three hyperparameters that could be optimized for the autoencoder.**

**Answer:**

YOUR ANSWER HERE

**Question 4: The autoencoder we implemented here used feed-forward/dense layers. Another way to improve the autoencoder reconstruction could be to use a different type of layer. What other type of layer might give better performance (e.g., convolutional, recurrent) and why?**

**Answer:**

YOUR ANSWER HERE